### Networks from other data sources

In [1]:
import pandas as pd
import networks.other_bio_networks as others
import datetime, pickle
from create_datasets import create_nx_datasets
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt


**1. Obtain other biological networks**

Obtain new biological networks for the AD gene set from different sources:
- 1. BioGRID: Protein-Protein Interactions (PPIs)
- 2. Human Reference Interactome (HuRI): PPIs
- 3. PPT-Ohmnet dataset from Stanford Network Analysis Project (SNAP): brain-specific PPIs
- 4. GIANT: functional network, brain-specific

In [ ]:
# BioGRID (PPI)
edgelist_biogrid = others.get_biogrid('data/AD_GDAs.txt')

# HuRI (PPI) - HuRI interactions were downloaded from HuRI webpage and then parsed to obtain the corresponding edgelist.
interactions_huri = pd.read_csv('data/other_networks/HuRI_download_interactions_dec_16_2021_11_54_03.csv', comment='#')
edgelist_huri = interactions_huri[['Interactor A Gene Name', 'Interactor B Gene Name']]
edgelist_huri = edgelist_huri[edgelist_huri['Interactor A Gene Name'] != edgelist_huri['Interactor B Gene Name']] # remove self loops
edgelist_huri.to_csv('data/other_networks/AD_HuRI_PPI.edgelist', sep='\t', index=None, header=False)

# PPT-Ohmnet (PPI, brain-specific)
network_snap = others.get_snap('data/AD_GDAs.txt')

# GIANT (functional network, brain-specific)
network_giant = others.get_giant('data/AD_GDAs.txt')

**2. Create graph datasets**

Create the correspoding graph-datasets for each newly obtained biological network.

In [ ]:
dataset = 'ADNI'
targets = ['PET', 'PETandDX']
networks = ['biogrid', 'huri', 'snap_brain', 'giant_brain', 'original']

for target in targets:
    for network in networks:

        outdir = f'data/graph_datasets/{target}'

        start_time = datetime.datetime.now()
        print()

        result_nodes = create_nx_datasets.main('data', dataset, target, 'AD', network, 'missense', None)
        print('Coding: number of missense variants per node')

        outfile = f'{outdir}/AD_PPI_{network}_missense.pkl'
        print('Resulting dataset saved at:', outfile)
        print()

        with open(outfile, 'wb') as f:
            pickle.dump(result_nodes, f)

        result_nodes_time = datetime.datetime.now()
        print('Processing time:', result_nodes_time - start_time)
        print('\n\n')

**3. Graph classification with GNNs**

We then evaluated and tested different GNNs in the framework called [GraphGym](https://github.com/snap-stanford/GraphGym) (You *et al.*, 2020).

Configuration and grid files employed are in the subdirectory [graphgym_files](graphgym_files).

Summarized results obtained by GraphGym and other models are in [results/GNNs_other_networks](results/GNNs_other_networks)

In [29]:
petdx_results

,Target,Dataset,Model,ACC,PRE,REC,F1,AUC,CM,TN,FP,FN,TP,Specificity,Sensitivity
0,PET,AD STRING,GNN GraphGym,0.5625,0.5625,1.0000,0.7200,0.6825,"[[0,14], [0, 18]]",0,14,0,18,1.000000,0.000000
1,PET,AD STRING,GNN GraphGym,0.5938,0.6190,0.7222,0.6667,0.7302,"[[6, 8], [5, 13]]",6,8,5,13,0.722222,0.428571
2,PET,AD STRING,GNN GraphGym,0.5938,0.6087,0.7778,0.6829,0.7143,"[[5, 9], [4, 14]]",5,9,4,14,0.777778,0.357143
3,PET,AD BioGRID,GNN GraphGym,0.6875,0.6818,0.8333,0.7500,0.6944,"[[7, 7], [3, 15]]",7,7,3,15,0.500000,0.833333
4,PET,AD BioGRID,GNN GraphGym,0.7188,0.7143,0.8333,0.7692,0.6865,"[[8, 6], [3, 15]]",8,6,3,15,0.571429,0.833333
5,PET,AD BioGRID,GNN GraphGym,0.7188,0.7368,0.7778,0.7568,0.6845,"[[9, 5], [4, 14]]",9,5,4,14,0.642857,0.777778
6,PET,AD HuRI,GNN GraphGym,0.7500,0.7500,0.8333,0.7895,0.7758,"[[9, 5], [3, 15]]",9,5,3,15,0.642857,0.833333
7,PET,AD HuRI,GNN GraphGym,0.7500,0.7273,0.8889,0.8000,0.7599,"[[8, 6], [2, 16]]",8,6,2,16,0.571429,0.888889
8,PET,AD HuRI,GNN GraphGym,0.7500,0.7500,0.8333,0.7895,0.8532,"[[9, 5], [3, 15]]",9,5,3,15,0.642857,0.833333
9,PET,AD PPT-Ohmnet\nBrain,GNN GraphGym,0.7500,0.7273,0.8889,0.8000,0.7778,"[[8, 6], [2, 16]]",8,6,2,16,0.571429,0.888889


In [ ]:
from scipy import stats

def statistics(df, baseline):
    result = pd.DataFrame(columns=['Random Forest', 'SVM Linear', 'SVM RBF', 'Logistic Regression', 'GNN GraphGym'])
    gnn_datasets = []
    for d in df['Dataset'].unique():
        print(d)
        tmp = df.loc[df['Dataset'] == d]
        gnn = tmp.loc[tmp['Model'] == 'GNN GraphGym']['AUC'].values
        gnn_datasets.append(gnn)
        for m in ['Random Forest', 'SVM Linear', 'SVM RBF', 'Logistic Regression', 'GNN GraphGym']:
            tmp_model = tmp.loc[tmp['Model'] == m]
            values = tmp_model['AUC'].values
            t2, p2 = stats.ttest_1samp(values, baseline, alternative='greater') # (a, popmean), mean of the underlying distribution of the sample is greater than the given population mean (popmean)
            t3, p3 = stats.ttest_ind(gnn, values, alternative='greater') # mean of the distribution underlying the first sample is greater than the mean of the distribution underlying the second sample.
            print(m, p2, p3)
            result[m] = [p2, p3]
        
        
    
    t4, p4 = stats.ttest_ind(gnn_datasets[0], gnn_datasets[1])
    print(p4)